# Исходные данные

В данной работе рассматривается индекс экономической свободы на основе статистических пакетов языка Python и облачной вычислительной среды Google Colabaratory, данные взяты с https://www.fraserinstitute.org/studies/economic-freedom-of-the-world-2016-annual-report 

Statsmodels-это модуль Python, который предоставляет классы и функции для оценки множества различных статистических моделей, а также для проведения статистических тестов и исследования статистических данных. Для каждого оценщика доступен обширный список статистики результатов. Полученные результаты проверяются на соответствие существующим статистическим пакетам, чтобы убедиться в их правильности

Категории

1: Размер правительства - По мере увеличения 
государственных расходов, налогообложения и размера
контролируемых государством предприятий принятие государственных
решений заменяется индивидуальным выбором, а экономическая
свобода сокращается.

2: Правовая система и права собственности - Защита лиц и
их законно приобретенной собственности является центральным элементом как
экономической свободы, так и гражданского общества. Действительно, это самая важная
функция правительства.

3: Надежные деньги - Инфляция подрывает ценность законно заработанных
заработных плат и сбережений. Таким образом, надежные деньги необходимы для защиты собственности
права. Когда инфляция не только высока, но и нестабильна, людям становится
трудно планировать будущее и, таким образом, эффективно использовать
экономическую свободу.

4: Свобода международной торговли - Свобода обмена—в самом
широком смысле, покупка, продажа, заключение контрактов и т. Д.—имеет
существенное значение для экономической свободы, которая уменьшается, когда свобода
обмена не включает предприятия и отдельных лиц в других
странах.

5: Регулирование - Правительства не только используют ряд инструментов для
ограничения права на международный обмен, но и могут развивать
обременительные правила, которые ограничивают право на обмен, получение кредита,
наем или работу для кого вы хотите, или свободное ведение вашего бизнеса.


# Добавление библиотек для работы с таблицами, статистики и вывода графиков 

In [ ]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn import svm
import graphviz
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline

Добавление файла CSV с которым мы работаем


In [ ]:
data = pd.read_csv('../input/economic-freedom/efw_cc.csv')
data.shape

In [ ]:
data.info()

In [ ]:
data.year.value_counts().sort_index().index

Получение вывода первых 10 строк таблицы + наименования колонок 

In [ ]:
data.head(10)

# Очистка данных

In [ ]:
data.isnull().sum()

Удаление Объектов С Более Чем 1242 Нулевыми Значениями
Здесь используются только те переменные, которые имеют более 1/3 ненулевых значений. Поскольку заполним значения NaN его медианой, этих столбцы, когда у них есть только несколько данных.

In [ ]:
data = data.loc[:, (data.isnull().sum(axis=0) <= 1242)]

Переименуем столбцы для лучшего понимания

In [ ]:
data.rename(columns={'year': 'YEAR',
                     'ISO_code': 'ISO_CODE',
                     'countries': 'COUNTRY',
                     'rank' :'RANK',
                     'quartile': 'QUARTILE',
                     'ECONOMIC FREEDOM': 'SCORE',
                     '1a_government_consumption': 'GOV_CONSUMPTION',
                     '1b_transfers': 'TRANSFERS',
                     '1c_gov_enterprises': 'GOV_ENTERPRISES',
                     '1d_top_marg_tax_rate': 'TOP_MARG_TAX_RATE',
                     '1_size_government': 'GOV_SIZE',
                     '2b_impartial_courts': 'IMPARTIAL_COURTS', 
                     '2c_protection_property_rights': 'PROTEC_PROP_RIGHTS',
                     '2d_military_interference': 'MILITARY_INTERF',
                     '2e_integrity_legal_system': 'INTEGRITY_LEGAL_SYST',
                     '2j_gender_adjustment': 'GENDER_ADJUSTMENT',
                     '2_property_rights': 'PROPERTY_RIGHTS',
                     '3a_money_growth': 'MONEY_GROWTH',
                     '3b_std_inflation': 'STD_INFLATION',
                     '3c_inflation': 'INFLATION',
                     '3d_freedom_own_foreign_currency': 'FOREIGN_CURRENCY',
                     '3_sound_money': 'SOUND_MONEY',
                     '4a_tariffs': 'TARIFFS',
                     '4c_black_market': 'BLACK_MARKET',
                     '4d_control_movement_capital_ppl': 'CONTROL_MOVEMENT',
                     '4_trade': 'TRADE',
                     '5a_credit_market_reg': 'CREDIT_MARKET_REG',
                     '5b_labor_market_reg': 'LABOR_MARKET_REG',
                     '5_regulation': 'REGULATION'}, inplace=True)

Заполняем недостающие значения с помощью медианы. Это не сильно повлияет на дисперсию, потому что функция будет получать медиану по странам, так что вероятность того, что значение будет правильным, выше.

In [ ]:
data.QUARTILE = data.QUARTILE.fillna(method='ffill')

num_names = data._get_numeric_data().columns

data[num_names] = data.groupby('ISO_CODE')[num_names].transform(lambda x: x.fillna(x.median()))

In [ ]:
data.isnull().sum()

Как мы видим, есть некоторые значения, не заполненные функцией. Это, вероятно, произошло потому, что ни в один год, соответствующей страны не было этой информации, чтобы взять медиану.

Но мы можем немного приблизиться, взяв медиану соответствующего квартиля. Учитывая, что страны одного квартиля имеют общие черты.

In [ ]:
data.QUARTILE = data.QUARTILE.astype('object')

data[['TRANSFERS','GOV_ENTERPRISES','PROTEC_PROP_RIGHTS','INTEGRITY_LEGAL_SYST','TARIFFS','BLACK_MARKET']] = data.groupby('QUARTILE')\
    [['TRANSFERS','GOV_ENTERPRISES','PROTEC_PROP_RIGHTS','INTEGRITY_LEGAL_SYST','TARIFFS','BLACK_MARKET']].transform(lambda x: x.fillna(x.median()))

# Средний рост глобальной экономической свободы

Создаем новую таблицу из исходной со столбцами Экономическая свобода и год, группируем по годам и средним значениям по всем странам.

In [ ]:
data_year=data[['SCORE','YEAR']]
data_year.groupby('YEAR').mean().plot(figsize=[9,9],)


Получили динамику изменения среднего значения индекса Экономической свободы за период с 1970-2016. Высокий уровень свободы в начале 70-х обусловлен движением Хиппи в США и Европе, а так же "Оттепели" и открытию железного занавеса в СССР. Упадок в 80-х можно обосновать тем, что движение Хиппи было подавлено, начались исламские революции на Ближнем Востоке, перевороты в Африканских странах, война в Афганестане, усиление коммунистической партии в Китае. Далее следует улучение ситуации в мире, до 2001 года. Что обусловлено террактом 11 сентября, войне в Персидсем заливе, серии террактов в России, что в целом, ввело ряд огранечений на экономическую свободу. И последний серьезный спад был в 2007-2009 годах, когда произошел Международный Финансовый Кризис, в следствии чего, было введено значительно мер, направленных на ужесточения регулирования финансового сектора. 

In [ ]:
data_geo=data[['SCORE','ISO_CODE','COUNTRY','YEAR']]
fig = px.scatter_geo(data_geo, locations="ISO_CODE", color="SCORE",
                     hover_name="COUNTRY",
                     animation_frame="YEAR",
                     projection="natural earth")
fig.show()

# Топ лучших и худших стран с экономической свободой

Набор данных содержит индекс для каждой страны с 1970 по 2016 год. Мы можем проверить, какие из них находятся на дне и в топе за 2016 год.

15 наименее Экономически свободных стран в 2016 году

In [ ]:
sns.set_palette(sns.dark_palette('purple',15, reverse=False))
sns.set_style('whitegrid')

top_15_16_least = data[data.YEAR==2016].sort_values(by='SCORE', ascending=False).tail(15)
top_15_16_least.plot('COUNTRY', 'SCORE', kind='bar', figsize=(14,8), rot=45)

plt.xlabel('СТРАНЫ')
plt.ylabel('УРОВЕНЬ')
plt.title('15 наименее Экономически свободных стран в 2016')

Большинство из 15 наименее развитых стран имеют примерно одинаковый балл. Только в Венесуэле, последней стране в рейтинге 2016 года, мы видим огромную разницу по сравнению с другими. Венесуэла переживает один из самых больших кризисов за последние несколько лет.

Другой заметный факт заключается в том, что большинство из них являются африканскими и азиатскими странами, за исключением Аргентины и Венесуэлы, которые расположены в Южной Америке.

15 наиболее Экономически свободных стран в 2016 году

In [ ]:
sns.set_palette(sns.dark_palette("seagreen",15, reverse=False))
sns.set_style('whitegrid')

top_15_2016 = data[data.YEAR==2016].sort_values(by='SCORE', ascending=False).head(15)
top_15_2016.plot('COUNTRY', 'SCORE', kind='bar', figsize=(14,8), rot=45)

plt.xlabel('СТРАНЫ')
plt.ylabel('УРОВЕНЬ')
plt.title('15 наиболее Экономически свободных стран')

Учитывая, что это топ-15 стран в 2016 году, мы не видим большой разницы между ними. Единственное замечание, которое можно сделать, состоит в том, что большинство этих стран из Европы или Азии, и у нас есть только одна из Африки (Маврикий) и одна из Южной Америки (Чили).

Но всегда ли эти страны были экономически свободны?

In [ ]:
names = top_15_2016['COUNTRY']
top_15 = data.loc[data['COUNTRY'].isin(names)]

sns.set_palette(sns.color_palette("colorblind",15))
sns.set_style('whitegrid')

fig, ax = plt.subplots()

for key, grp in top_15.groupby(['COUNTRY']):
    ax = grp.plot(ax=ax, kind='line', x='YEAR', y='SCORE', label=key, figsize=(20,10), linewidth=2.5)
    
plt.xlim((1970, 2016))
plt.xlabel('ГОД')
plt.ylabel('РЕЙТИНГ')
plt.title('Динамика рейтинга с 1970 по 2016')

На этом графике видно, что большинство из этих стран всегда имели высокий индекс экономической свободы. Некоторые из них имели большие колебания в течение многих лет, вероятно, вызванные правительствами или историческими событиями.

За исключением Чили, это единственная южноамериканская страна среди 15 лучших стран в 2016 году. И самое интересное, что к середине 70-х годов Чили была страной 4-го квартиля, что связано с приходом к власти Аугусто Пиночета в результате военного переворота.

## BRICS и Казахстан

### Как только мы заговорили о развитии стран, были  взяты страны БРИКС и сравнили их с Казахстаном.

Для тех, кто не знаком, БРИКС-это аббревиатура для развивающихся экономик Бразилии, России, Индии, Китая и Южной Африки. Вместе они составляют около 25% всей суши мира и более 40% его населения.

In [ ]:
briccs_names = ['Brazil', 'Russia', 'India', 'China', 'Kazakhstan', 'South Africa']
briccs = data.loc[data['COUNTRY'].isin(briccs_names)]

sns.set_palette(sns.color_palette("bright",6))
sns.set_style('whitegrid')

fig, ax = plt.subplots()

for key, grp in briccs.groupby(['COUNTRY']):
    ax = grp.plot(ax=ax, kind='line', x='YEAR', y='SCORE', label=key, figsize=(18,10), linewidth=2.5)
    
plt.xlim((2000, 2016))
plt.legend(loc='lower right')
plt.xlabel('Годы')
plt.ylabel('Уровень')
plt.title('Уровень BRICCS с 2000 по 2016 гг')

На 2016 год, Казахстан занимает более высокий уровень Экономической свободы, чем какая либо из стран BRICS, это возможно связанно с неверной статистикой, так как данные появляются с середины 2000-х годов, значения до этого заменены медианой. Подобное, наблюдается и у России до 90-го года, что обуславливается тем, что до этого периода был СССР и данных не было.





Среди стран БРИКС все страны имеют индекс от 6,0 до 7,0, за исключением Бразилии, которая остается ниже 6,0. В Казахстане значение выше 7,0.


# Получение описательной статистики по вссем колонкам с числовыми значениями

In [ ]:
data.describe()

# Что влияет на Экономическую свободу?

Построим тепловую корреляционную таблицу по всем столбцам

In [ ]:
data_num = data._get_numeric_data()
data_cor = data_num.corr()
sns.set(font_scale=1.4)
plt.figure(figsize=(13,13))
sns.heatmap(data_cor,  square=True, cmap='coolwarm_r')

Для наглядности построим тепловую корреляционную матрицу на основе столбцлв: "Экономической свободы", "Размера правительства", "Правовая система", "Надежные деньги", "Свободная торговля", "Регулирование"

In [ ]:
data_hit=data[['SCORE', 'GOV_SIZE', 'PROPERTY_RIGHTS', 'SOUND_MONEY', 'TRADE', 'REGULATION']]
sns.set(font_scale=1.3)
x,ax=plt.subplots(figsize=(12,12))
sns.heatmap(data_hit.corr(),cbar=True,annot=True,fmt='.2f',square=True)

In [ ]:
data_hit.describe()

Получив корреляционную таблицу, приходим к выводу, что на индекс экономической свободы сильнее всего влияют: индекс биржевой торговли, надежные деньги и регулирование. При этом размер правительства слабо влияет на уровень экономической свободы.

Построим графики 5 категорий


In [ ]:
g = sns.PairGrid(data_hit)
g.map(sns.scatterplot)

Получим автоматический дашборд

In [ ]:
profile = ProfileReport(data_hit)

In [ ]:
profile.to_notebook_iframe()

#  Построение модели

Строим регрессионную модель, где Y - Уровень Экономической свободы, а за X принимаем массив из столбцов ["Размер правительства", "Права и Свободы", "Надежность денег", "Свободную торговлю" и "Регулирование"

In [ ]:
Y = data['SCORE']
X = data[['GOV_SIZE', 'PROPERTY_RIGHTS', 'SOUND_MONEY', 'TRADE', 'REGULATION']]

Строим модель и таблицу

In [ ]:
X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)
print(model.summary())

Проверим на гетероскедастичность Тестом Бреуша Пагана Годфри И тест Уайта

In [ ]:
print("""
Критическое значение, 
p-значение теста множителя Лагранжа, 
статистика гипотезы о том, что дисперсия ошибки не зависит от x, 
p-значение для f-статистики
""" , (sm.stats.diagnostic.het_breuschpagan(Y, X)))

In [ ]:
print("""
Критическое значение, 
p-значение теста множителя Лагранжа, 
статистика гипотезы о том, что дисперсия ошибки не зависит от x, 
p-значение для f-статистики
""" ,sm.stats.diagnostic.het_white(Y, X))

По тестам на Гетероскедастичность, мы получили значения Р выше критических значений, что говорит, что нулевая гипотиза подтверждается, что гетероскедастичность отсутсвует

Посчитаем Коэффициент инфляции, эмпирическое правило гласит, что если ВИФ больше 10, то присутствует мультиколлениарность, а это говорит о том, что необходимо убрать лишний параметр и пересчитывать 

In [ ]:
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns
vif = vif.drop([0])

In [ ]:
vif

In [ ]:
print(model.params)

$$SCORE = Const + w_1 \times X_1 + w_2 \times X_2 + w_3 \times X_3 + w_4 \times X_4 + w_5 \times X_5$$


Общее уравнение получается: 
$$SCORE = 0,177167 + 0,186342 \times GOVSIZE + 0,201158 \times PROPERTYRIGHTS + 0.190776 \times SOUND_MONEY + 0.198816 \times TRADE +  0.199991 \times REGULATION $$

Итог:
R квадрат и R квадрат перещитанный 0,968, что говорит о высокой точности модели, F-статистика незначительна, можно предположить, что вероятность ошибки регрессии незначительна. При этом, имеется смещение и высокий куртосис больше 3, тест Харке-Бера значительно отличен от 0, поэтому отклоняем нулевую гепотизу о нормальном распределении. Критерий Дарбин-Вотсана близок к 2, значит, что нет автокорреляции, t статистика говорит о том, чо все переменные значимы.

# Метод Опорных Векторов

Задача регрессии состоит в том, чтобы найти функцию, аппроксимирующую отображение из входной области в действительные числа на основе обучающей выборки. Итак, давайте теперь погрузимся глубоко и поймем, как на самом деле работает SVR.

Метод Опорных Векторов или SVM (от англ. Support Vector Machines) — это линейный алгоритм используемый в задачах классификации и регрессии. Данный алгоритм имеет широкое применение на практике и может решать как линейные так и нелинейные задачи. Суть работы “Машин” Опорных Векторов проста: алгоритм создает линию или гиперплоскость, которая разделяет данные на классы.


## Как SVM находит лучшую линию

Алгоритм SVM устроен таким образом, что он ищет точки на графике, которые расположены непосредственно к линии разделения ближе всего. Эти точки называются опорными векторами. Затем, алгоритм вычисляет расстояние между опорными векторами и разделяющей плоскостью. Это расстояние которое называется зазором. Основная цель алгоритма — максимизировать расстояние зазора. Лучшей гиперплоскостью считается такая гиперплоскость, для которой этот зазор является максимально большим.

Этот метод назван опорный вектор регрессии (SVR). Это модель построена с использованием опорного метода классификации (описанного выше) и зависит только от подмножества данных для обучения, т.к. штрафная функция при построении модели не обращает внимания о том, что точка лежит за пределами края.
Другая версия SVM хорошо известна как метод наименьшего квадратного опорного вектора (LS-SVM) был предложен Suykens и Vandewalle.
В машинном обучении, метод опорных векторов (SVMs) – контролируемое обучение моделей с использование схожих алгоритмов для анализа данных и распознавания шаблонов.
Данный метод используется для задач классификации и регрессионного анализа. Основной метод опорных векторов принимает набор входных данных и прогнозирует для каждого данного входа одну из двух возможных форм выхода.
Благодаря такому процессу, данный метод случайным является бинарным линейным классификатором.
Учитывая набор обучающих наблюдений (обучающую выборку), каждое из которых помечена как принадлежащая к одной из двух категорий, алгоритм обучения метода опорных векторов строит модель, которая определяет новые наблюдения в одну из категорий.
Модель метода опорных векторов – отображение данных точками в пространстве, так что между наблюдениями отдельных категорий имеется разрыв и он максимален.
Затем новые наблюдения отобразятся в том же пространстве и будут относиться к одной из категорий в зависимости от того, по какую сторону разрыва они отобразились.

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Обучим модель, проверим несколько функций регрессии
1.   Радиально-базисная функция
2.   Линейная
3.   Полиноминальная



Радиальные базисные функции (Radial basis functions) (RBF) представляют собой набор методов жесткой интерполяции; это означает, что поверхность должна проходить через каждое измеренное опорное значение. Существует пять различных базисных функций:

* Плоский сплайн
* Сплайн с натяжением
* Полностью регуляризованный сплайн
* Функция мультиквадриков
* Функция Обратные мультиквадрики

Каждая базисная функция имеет различную форму и позволяет получать разные интерполированные поверхности. Методы РБФ – это специальный случай сплайнов.

Концептуально, радиальные базисные функции напоминают размещение резиновой мембраны на измеренные опорные точки и одновременно уменьшение общей кривизны поверхности. Выбор базисной функции определяет то, как резиновая мембрана будет расположена между значениями. В следующей диаграмме концептуально показано нанесение поверхности радиальной базисной функции на серию опорных точек высот. Обратите внимание, что поверхность в поперечном сечении проходит через значения данных.

Часто используемые радиально-базисные функций включают в себя (
r
=
‖
x
−
x
i
‖
$${\displaystyle r=\left\|\mathbf {x} -\mathbf {x} _{i}\right\|}):$$

Функция Гаусса:
ϕ
(
r
)
=
e
−
(
ε
r
)
2
$${\displaystyle \phi \left(r\right)=e^{-\left(\varepsilon r\right)^{2}}}$$
Мультиквадратичная:
ϕ
(
r
)
=
1
+
(
ε
r
)
2
$${\displaystyle \phi \left(r\right)={\sqrt {1+\left(\varepsilon r\right)^{2}}}}$$
Обратная квадратичная:
ϕ
(
r
)
=
1
1
+
(
ε
r
)
2
$${\displaystyle {\displaystyle \phi \left(r\right)={\dfrac {1}{1+\left(\varepsilon r\right)^{2}}}}}$$
Обратная мультиквадратичная:
ϕ
(
r
)
=
1
1
+
(
ε
r
)
2
$${\displaystyle \phi \left(r\right)={\dfrac {1}{\sqrt {1+\left(\varepsilon r\right)^{2}}}}}$$
Полигармонический сплайн:
ϕ
(
r
)
=
r
k
,	k	=1,3,5,…	ϕ
(
r
)
=
r
k
ln⁡
(
r
)
,	k	=2,4,6,… 
$${\displaystyle {\begin{aligned}\phi \left(r\right)&=r^{k},&k&=1,3,5,\dotsc \\\phi \left(r\right)&=r^{k}\ln \left(r\right),&k&=2,4,6,\dotsc \end{aligned}}}$$
Тонкий сплайн пластины (специальный полигармонический сплайн):
ϕ
(
r
)
=
r
2
ln
⁡
(
r
)
$${\displaystyle \phi \left(r\right)=r^{2}\ln \left(r\right)}$$

Разделим данные на переменные и отклик

In [ ]:
Y = data['SCORE']
X = data[['GOV_SIZE', 'PROPERTY_RIGHTS', 'SOUND_MONEY', 'TRADE', 'REGULATION']]

Разделим на тренировочные и тестовые выборки


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.4, random_state=0)

### Обучим модель опорных векторов на тестовой выборке, где стандартной функцией, принимаются Радиально Базисные Функции

In [ ]:
regr_1 = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr_1.fit(X_train, y_train)

Получим результат точности, и проверим на тестовой выборке

In [ ]:
regr_1.score(X_test, y_test, sample_weight=None)

Получено значение давольно высокое, наша модель с 95,3% точностью определяет Рейтинг


### Обучим модель на основе линейной модели
По сути это является проверкой обычной линейной регрессии, что была выше на тренирочных и тестовых значениях

In [ ]:
regr_2 = make_pipeline(StandardScaler(), SVR(kernel='linear', C=1.0, epsilon=0.2))
regr_2.fit(X_train, y_train)

In [ ]:
regr_2.score(X_test, y_test, sample_weight=None)

Полученое значение ниже, чем было в первой модели. Для первой модели линейной регрессии значение было 96,8%, но на тесте мы получили значение 94,3% 

### Обучим модель на основе полиноминальной функции
Степень берем эмпирически 3


In [ ]:
regr_3 = make_pipeline(StandardScaler(), SVR(kernel='poly', C=1.0, epsilon=0.2, degree=3))
regr_3.fit(X_train, y_train)

In [ ]:
regr_3.score(X_test, y_test, sample_weight=None)

Полученое значение значительно ниже, чем было в моделях выше. Так что отклоняем даанный вариант.

Из 3 предложенных функций, подходит больше всего 1, точность выше, чем при подходе обычной линейной регрессии, получим датафрейм с предсказанием функции

# Вывод
